## Введение

Широко известно, что для машинного обучения используется Python. Но также известно, что Python уступает по своему быстродействию *компилируемым* языкам типа C++, C# или Kotlin. Поэтому не всегда понятно, почему для задач ML, где используются тяжеловесные модели, которые должны отвечать быстро, используется интерпретируемый Python.

На этот вопрос есть два ответа:
* удобный синтаксис для операций над векторами
* скорость векторизованных операций

Библиотека NumPy, которую мы разберём на этом семинаре, позволяет многократно оптимизировать работу с векторами по памяти и по времени. Если не вдаваться глубоко в подробности работы векторов NumPy, то они
* Работают быстро благодаря переходу исполнения от Python к C++
* Внутри операции ускоряются за счет векторизации (распараллеливание на уровне процессора или т.н. AVX-инструкции)
* Занимают меньше памяти благодаря типизации - к примеру, по умолчанию векторы заполняются целыми либо вещественными числами. Также как и обычные списки, векторы NumPy могут содержать данные разных типов, но по умолчанию в них выделяется память под элементы одного (заранее выбранного) типа.

С библиотекой numpy также связаны другие библиотеки, использующиеся в машинном обучении и анализе данных. На этом семинаре помимо numpy разберём pandas и matplotlib/seaborn.

## Numpy

In [1]:
import numpy as np  # np - общепринятый алиас для библиотеки numpy

### Инициализация numpy-массивов

In [2]:
_ = np.zeros(5)  # [0] * 5
_ = np.ones(10)  # [1] * 10
_ = np.arange(10)  # [i for i in range(10)]
_ = np.arange(5, 10, 2)  # [i for i in range(5, 10, 2)]
_ = np.arange(1, 10, 0.1)  # шаг с плавающей точкой
_ = np.linspace(1, 10, 100)  # почти то же самое, только указываем не шаг, а число точек
_ = np.eye(3)  # создает единичную матрицу 3x3

In [3]:
x = np.zeros((10, 3, 8))  # инициализация массива 10х3х8
print(x.shape)  # свойство, отвечающее за размеры массива вдоль разных осей, называется shape

(10, 3, 8)


In [4]:
python_list = np.zeros((3, 3)).tolist()  # можем получить как питоновский список из массива
np_array = np.array(python_list)  # так и numpy-массив из списка

### Векторизованные операции

Сделаем два np-массива

In [5]:
a = np.arange(1, 6)
b = a[::-1]

# Обратное индексирование: пройдём весь массив a[:] с индексацией -1
# a[-1] == a[len(a) - 1];
# a[-2] == a[len(a) - 2];
# ...
# a[-len(a)] = a[len(a) - len(a)];

print(a)
print(b)

[1 2 3 4 5]
[5 4 3 2 1]


Теперь умеем так:

In [6]:
print(a + b)
print(a * b)
print(a / b)
print(a - 10)
print(a * 0.5)
print(a / 0.123)

[6 6 6 6 6]
[5 8 9 8 5]
[0.2 0.5 1.  2.  5. ]
[-9 -8 -7 -6 -5]
[0.5 1.  1.5 2.  2.5]
[ 8.1300813 16.2601626 24.3902439 32.5203252 40.6504065]


Что произойдёт, если попытаться сложить np.ones(3) и np.ones(5)?

In [7]:
print(np.sum(a), a.sum())
print(np.mean(a), a.mean())
print(np.min(a), a.min())
print(np.max(a), a.max())

# у np-массива нет функции медианы, но зато такая функция есть в модуле numpy
# как думаете, почему?
print(np.median(a))

15 15
3.0 3.0
1 1
5 5
3.0


Оценим скорость векторизованных операций

In [8]:
def rand_mul(a):
    res = [0] * len(a)  # сэкономим время, заранее выделив память
    rnd = np.random.random(len(a))
    for i in range(len(a)):
        res[i] = a[i] * rnd[i]
    return res


a = np.arange(10000)

In [9]:
%timeit rand_mul(a)

18.6 ms ± 712 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%timeit a * np.random.random(10000)

60.4 µs ± 6.73 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Примерно в **250** раз меньше времени на 1 операцию

Есть полезная функция `np.vectorize`, в которую захочется пихать любой python-код. На самом деле есть ограничения на код, который можно распараллелить на уровне процессора.

Найдите отличия между тремя циклами:

Цикл 1 (можно векторизовать)
```
for i in range(n):
    v[i] += a[i]
```

Цикл 2 (можно векторизовать)
```
for i in range(1, n - 1):
    v[i + 1] += a[i - 1]
```

Цикл 3 (нельзя векторизовать)
```
for i in range(3, n):
    v[i] = v[i - 1] + v[i - 2] + v[i - 3]
```

### Индексация. Маски. Функции по индексам

Разберём несколько простых примеров на индексацию.

In [11]:
a = np.array([[1, 2, 3], [4, 5, 6]])

In [12]:
print(a[0, 0]) # вместо a[0][0]
print(a[:, ::-1])
print(a[:, ::2])

1
[[3 2 1]
 [6 5 4]]
[[1 3]
 [4 6]]


Логические операции, будучи применёнными к np.array, возвращают **маску** - другой массив, содержащий значения типа `bool`, которые показывают истинность или ложность проверяемого выражения для **каждого** элемента

In [13]:
print(a % 2 == 0) # что вернёт a % 2?
print(a > 3)

[[False  True False]
 [ True False  True]]
[[False False False]
 [ True  True  True]]


Маски используются для итерации по исходному массиву:

In [14]:
print(a[a % 2 == 0])
print(a[a > 3])

[2 4 6]
[4 5 6]


Условия масок можно комбинировать через `&` и `|`, в этом случае каждое условие ограничивается скобками

In [15]:
print((a >= 2) & (a <= 4))

print(a[(a % 2 == 0) & (a > 3)])
print(a[(a % 2 == 1) | (a <= 3)])

[[False  True  True]
 [ True False False]]
[4 6]
[1 2 3 5]


## Pandas

In [16]:
import pandas as pd  # pd - общепринятый алиас для библиотеки pandas

### Датафреймы, Ряды

Основная структура, используемая в pandas - DataFrame. По сути представляет из себя удобную таблицу, с которой можно работать и как с таблицей, и как с numpy-массивом

In [17]:
_ = pd.read_csv('path/to/file.csv')  # можем считать из csv-файла
_ = pd.read_excel('path/to/excel.xlsx')  # можем считать из excel-файла (но с некоторыми ограничениями)

_ = pd.DataFrame(data=[(1, 'a'), (3, 'b'), (2, 'c')], columns=['num', 'char'])  # можем инициализировать из обычных списков
_ = pd.DataFrame(data={'num': [1, 3, 2], 'char': ['a', 'b', 'c']})  # можем инициализировать из словаря
                                                                    # ключ - название колонки, значение - элементы столбца

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/file.csv'

Внутри датафрейм состоит из т.н. рядов, как вдоль строк, так и вдоль столбцов. Ряд - это специальный тип `Series`, который является полноценным расширением numpy-массивов, при этом предоставляя разные табличные функции.

In [ ]:
!gdown 1uOH6YMfSPhGAcneQQST_OtrbmdME9g25 --out dataset.csv

In [ ]:
df = pd.read_csv('dataset.csv')

In [ ]:
df.head()  # отображает содержимое первых пяти строк

In [ ]:
df.sample(5)  # отображает содержимое !СЛУЧАЙНЫХ! 5 строк

Если название колонки соответствует допустимому питоновскому имени, то к колонке можно обращаться через точку

In [ ]:
df.city.sample(5)

В общем случае к колонкам можно обращаться через индексацию (квадратные скобки)

In [ ]:
df['floor'].sample(5)

In [ ]:
type(df.city)

### Индексация

Получить значение из ячейки датафрейма по строке и столбцу - нетривиальная задача. Например, если мы просто напишем `df[0, 0]`, то мы получим ошибку

In [ ]:
df[0, 0]

В pandas у ячеек есть две очевидные координаты - строка и столбец. Если хотим указать, по какому столбцу мы хотим искать ячейку, нам надо использовать название этого столбца.

У расположения по строке есть две основные опции:
- Номер строки (привычный нам индекс элемента в массиве)
- Индекс строки

Индексом в датафрейме называется специальный массив, который хранит набор уникальных обозначений для каждой строки. По умолчанию индекс совпадает с нумерацией строк, но некоторые операции над датафреймами меняют индекс, из-за чего может возникнуть путаница.

Кажется, что всё просто, да?

In [ ]:
df[0, 'city']

Оказывается, что для индексации по датафрейму используются два отдельных свойства: `loc` и `iloc`. При этом `loc` использует стандартный индекс pandas, а `iloc` - привычную нам индексацию от 0 до `n-1`.

In [ ]:
print(df.loc[0, 'city'], df.iloc[0, 0])

In [ ]:
print(df.loc[0, 0])  # loc не умеет работать с целочисленными индексами

In [ ]:
df.iloc[0, 'city']  # в то время как iloc не умеет работать со строковыми индексами

В остальном у индексации по датафреймам есть те же возможности, что и у индексации по numpy-массивам - маски и применение этих масок

In [ ]:
len(df.loc[df.city == 'Пермь'])  # сколько строк в датафрейме имеют значение "Пермь" в колонке city?

In [ ]:
df.loc[df.floor.isna()].sample(5)  # случайные 5 строк из всех, имеющих NaN в колонке floor

In [ ]:
len(df.loc[df.total_square > 200]) / len(df)  # какая доля объявлений предлагает площадь собственности от 200 м^2?

### Разные преобразования

Что можно делать со столбцами в датафрейме?
- Можно применять к ним любые математические операции
- Можно сложить, умножить и т.д. два столбца в датафрейме, если они обе числовые
- К столбцу можно применить любую специфическую функцию
- Создавать новые столбцы, удалять, переименовывать
- Модифицировать столбцы по слайсу

In [ ]:
df = pd.DataFrame(data={
    'num1': [0.1583017 , -0.54775732,  1.49827325,  1.20037232, -1.29960477, -1.7931545 ],
    'num2': [4, 0, 5, 15, -13, 10],
    'str1': ['a', 'b', 'c', 'd', 'e', 'f'],
    'str2': ['z', 'y', 'x', 'w', 'v', 'u'],
    'to_delete': ['a1', 'b2', 'c3', 'd4', 'e5', 'f6'],
    'to_rename': ['z1', 'y2', 'x3', 'w4', 'v5', 'u6'],
})

In [ ]:
df.num1 *= 3  # умножим все значения в столбце num1 на 3
df.head()

In [ ]:
df['num3'] = df.num1 + df.num2  # создадим новый столбец num3, являющийся суммой столбцов num1 и num2
df.head()

In [ ]:
df_new = df.drop(columns=['to_delete']).rename(columns={'to_rename': 'after_rename'})
# функции модификации типа drop/rename и т.д. возвращают новый датафрейм
df_new.head()

In [ ]:
"""
'abcde' -> 'a-b-c-d-e'
"""
def f(s: str) -> str:
    return '-'.join(list(s))


# функция apply есть как у объекта Series, так и у DataFrame
# соответственно, использовать для модификации можно как отдельную колонку, так и все колонки сразу
# в этом примере df_new.str1 + df_new.str2 создают новый объект Series, к которому apply применяется без проблем
df_new['new_str'] = (df_new.str1 + df_new.str2).apply(f)
df_new.head()

Что можно делать с самим датафреймом?
- Присоединять к нему другой датафрейм на основе какой-то колонки
- Делать сцепку (конкатенировать) два датафрейма как вдоль строк, так и вдоль колонок

In [ ]:
df_joint1 = pd.concat([
    df[['num1', 'num2', 'num3', 'str1', 'str2']],
    df_new[['num1', 'num2', 'num3', 'str1', 'str2']]
])
df_joint1.sample(10)

Попробуйте применить функцию `reindex` к датафрейму `df_joint1` и посмотрите, что изменится

In [ ]:
df_joint2 = pd.merge(df, df_new, on=['str1'], suffixes=['_left', '_right'])  # аналог inner join в SQL
df_joint2.head()

Отдельным пунктом внутри преобразований датафрейма хочется вынести использование библиотеки tqdm. Сама по себе библиотека tqdm очень простая, её задача - отобразить полосу прогресса какого-либо цикла

In [ ]:
from tqdm import tqdm

for i in tqdm(range(10)):
    pass

Если ваша задача предполагает работу с большим датафреймом, вы наверняка хотите знать, какая часть преобразования колонки уже выполнилась и сколько еще надо ждать. Делается это очень просто:

In [ ]:
tqdm.pandas()  # Обновляет типы DataFrame и Series, добавляя им метод progress_apply

In [ ]:
df_new['dash_cnt'] = df_new.new_str.progress_apply(lambda s: sum([1 if c == '-' else 0 for c in s]))
df_new.head()

## Визуализация (matplotlib, seaborn)

In [ ]:
from matplotlib import pyplot as plt  # plt - общепринятный алиас для библиотеки matplotlib
import seaborn as sns  # sns - общепринятый алиас для библиотеки seaborn

### Графики

Как сделать графики matplotlib красивыми? Для этого нужен простой советский `sns.set_theme()`

In [ ]:
sns.set_theme()

Что нам может понадобиться для визуализации?
- Построение каких-либо непрерывных графиков

In [ ]:
x = np.linspace(-10, 10, 100)  # зададим 100 точек от -10 до 10
y = 1/(1+np.exp(-x))  # посчитаем функцию f(x) от этих точек

_ = plt.plot(x, y)

- Визуализация точек в пространстве (возможно, в какой-либо проекции)

In [ ]:
x = np.random.randn(100)
y = np.random.randn(100)

_ = plt.scatter(x, y)

- Гистограммы, просто чтобы визуализировать какую-нибудь статистику

In [ ]:
_ = plt.hist(x)

### Комбинация графиков

In [ ]:
x = np.linspace(-10, 10, 100)  # зададим 100 точек от -10 до 10
y1 = 1/(1+np.exp(-x))  # посчитаем функцию f(x) от этих точек
y2 = np.arctan(x) / np.pi + 0.5  # посчитаем функцию g(x) от этих точек

_ = plt.plot(x, y1)  # синий график
_ = plt.plot(x, y2)  # оранжевый график

In [ ]:
x1, x2 = np.random.randn(1000) * 0.3 - 2, np.random.randn(1000) * 2 + 3
y1, y2 = np.random.randn(1000) * 0.5 - 3, np.random.randn(1000) * 0.8 + 1

plt.scatter(x1, y1)
plt.scatter(x2, y2)

In [ ]:
_ = plt.hist(x1, alpha=0.75, label='x1')  # alpha - прозрачность конкретной гистограммы
_ = plt.hist(x2, alpha=0.75, label='x2')
_ = plt.hist(y1, alpha=0.75, label='y1')
_ = plt.hist(y2, alpha=0.75, label='y2')

Если хотим разные графики - после каждого построения нужно вызвать `plt.show()`

In [ ]:
plt.hist(x1, alpha=0.5)
plt.hist(x2, alpha=0.5)
plt.show()

plt.hist(y1, alpha=0.75)
plt.hist(y2, alpha=0.75)
plt.show()

Eсли у одной и той же точки есть несколько допустимых измерений, то `sns.relplot` покажет доверительный интервал

In [ ]:
x = np.linspace(-10, 10, 100)

y0 = np.arctan(x) + np.random.randn(100) * 0.05 # функция с шумом 1
y1 = np.arctan(x) + np.random.randn(100) * 0.1 # функция с шумом 2
y2 = np.arctan(x) + np.random.randn(100) * 0.3 # функция с шумом 3
y = np.arctan(x)

X = np.hstack([x, x, x])
Y = np.hstack([y0, y1, y2])

sns.lineplot(
    x=x, y=y
).set(title='clear f(x)')
sns.relplot(
    x=X, y=Y, kind="line",
).set(title='noised f(x)')